prepare dataset

In [2]:
import os
import cv2
from utils.faceDetection import face_detector

alfw_dir = 'dataset/lfw'

for fld_name in os.listdir(alfw_dir):
    fld_path = os.path.join(alfw_dir, fld_name)
    for img_name in os.listdir(fld_path):
        img_path = os.path.join(fld_path, img_name)
        img = cv2.imread(img_path)
        os.remove(img_path)
        face_locs = face_detector(img)
        if len(face_locs[0]) == 1:
            (x,y,w,h) = face_locs[0][0]
            face_croped = img.copy()[y:y+h,x:x+w]
            cv2.imwrite(img_path, face_croped)

evaluate

In [ ]:
from evaluate_on_lfw import lfw
from utils.featureExtraction import feature_extraction
import cv2
import numpy as np


pair_filename_path = 'dataset/lfw/pairs.txt'
lfw_dir = 'dataset/lfw/lfw'
thresholds = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

pairs = lfw.read_pairs(pair_filename_path)
paths, actual_issame = lfw.get_paths(lfw_dir, pairs)

print('Embedding on LFW images')
embeddings = []
for path in paths:
    img0 = cv2.imread(path[0])
    embedding0 = feature_extraction(img0)
    img1 = cv2.imread(path[1])
    embedding1 = feature_extraction(img1)
    embeddings += (embedding0, embedding1)

np.save('dataset/lfw/embedding.npy', embeddings)

for threshold in thresholds:
    print('---------------------------------------')

    accuracy, precision, recall = lfw.evaluate(embeddings, actual_issame, threshold)

    f1_score = 0 if (precision+recall==0) else 2*precision*recall/(precision+recall)
        
    print('Threshold: %3.5f' % threshold)

    print('Accuracy: %3.5f' % accuracy)
    print('Precision: %3.5f' % precision)
    print('Recall: %3.5f' % recall)
    print('F1 score: %3.5f' % f1_score)
    